# Understanding dropouts

## Introduction
Le **surapprentissage** est un problème majeur dans l'entraînement des réseaux de neurones profonds. Une technique efficace pour le combattre est le **Dropout**.

Dans ce notebook, nous allons :
1. Mais le dropout, c'est quoi ?
2. Exemple d'application de la technique du Dropout.
3. Comparer les performances d'un réseau avec et sans Dropout.
4. Présenter cette technique sur d'autres types de modèles

<div style="background-color: #d4edda; color: black; padding: 10px; border-radius: 5px;">
Plan d'action, on va faire un exemple de dropout et ensuite on va venir comparer aux autres techniques de régularisation et enfin on va venir les combiner et montrer que le dropout se combine très bien avec du L2 ou norma par lots par exemple dans notre cas (MNIST).

Ensuite on va venir faire un état de l'art des différents réseaux de neuronnes qui sont efficaces avec le dropouts ou pas et conclure quant à cette technique et son utilisation sur les modèles de deep.
</div>

***
## 1. Mais le dropout, c'est quoi ?

### Un peu d'histoire
Avant l’introduction du Dropout, plusieurs approches avaient déjà exploré l’idée d’ajouter du bruit aux réseaux de neurones pour améliorer leur généralisation. **Hanson (1990)** a proposé la **Stochastic Delta Rule**, qui injectait du bruit dans l’apprentissage des poids pour limiter le surapprentissage. **Bishop (1995)** a démontré que **perturber les entrées** d’un modèle pouvait être interprété comme une forme de régularisation bayésienne. **LeCun et al. (1998)** ont testé l’**ajout de bruit** dans les activations des neurones pour limiter la dépendance excessive aux données d'entraînement. D’autres travaux, comme ceux de **Hinton & Nowlan (1992)** et **Neal (1995, 2001)**, ont étudié l’utilisation de **distributions probabilistes sur les poids et les activations** afin d’améliorer la généralisation des modèles.

Le **Dropout** a été introduit en **2014** par **Srivastava et al.** dans leur article **"Dropout: A Simple Way to Prevent Neural Networks from Overfitting"**. Cette technique a été développée pour pallier le problème du surapprentissage dans les réseaux de neurones profonds. Avant son introduction, les méthodes classiques de régularisation comme la **pénalisation L2** et le **early stopping** étaient couramment utilisées, mais elles ne suffisaient pas toujours à éviter l'adaptation excessive aux données d'entraînement.

L'idée principale derrière le Dropout était inspirée de l'**apprentissage par ensembles**, où plusieurs modèles indépendants sont combinés pour améliorer la généralisation. Toutefois, entraîner et stocker plusieurs réseaux de neurones profonds était **coûteux en calcul**. Srivastava et son équipe ont alors cherché un moyen d'obtenir un effet similaire au **model averaging**, mais de manière bien **plus efficace**.


### Fonctionnement du Dropout

Leur solution a été de **perturber l'apprentissage en désactivant aléatoirement des neurones à chaque itération**, forçant ainsi chaque neurone à apprendre des représentations **plus robustes** sans dépendre excessivement de neurones spécifiques. En empêchant la formation de **co-adaptations trop spécialisées**, cette approche a conduit à des modèles généralisant mieux aux données non vues. De plus, lors de la phase de **test**, tous les **neurones sont activés** mais leurs **poids sont ajustés** pour compenser les désactivations précédentes, simulant ainsi un moyennage implicite d'un grand nombre de **réseaux plus petits entraînés en parallèle**.


### Impact sur l’apprentissage des réseaux de neurones
Le Dropout a plusieurs effets bénéfiques sur l'apprentissage :
- **Réduction du surapprentissage** : en empêchant les neurones de trop s’adapter aux données d’entraînement.
- **Amélioration de la robustesse** : chaque neurone doit apprendre des représentations plus générales, car il ne peut pas compter sur d’autres neurones spécifiques.
- **Effet d’ensemble (ensemble learning)** : en échantillonnant différents sous-réseaux à chaque itération, le modèle final se comporte comme une combinaison de plusieurs réseaux différents, ce qui améliore la généralisation.


***
## 2. Exemple d'application de la technique du Dropout.

### Présentation du modèle 

Ca sera plus simple pour tout le monde si je reprends un modèle que l'on connait. On va tout simplement utiliser le modèle que nous avions défini au TD de deep learning sur le jeu de données FASHION-MNIST. Nous partirons de là et nous appliquerons la technique du dropout pour vraiment comprendre l'interêt de cette technique.

[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) est un jeu de données contenant des images d'articles de Zalando, composé d'un ensemble d'entraînement de 60 000 exemples et d'un ensemble de test de 10 000 exemples. Chaque exemple est une image en niveaux de gris de 28x28 pixels, associée à une étiquette parmi 10 classes. 

<img src="img/fashion-mnist-small.png">

In [ ]:
# Importation des librairies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Fonction d'entraînement avec suivi de l'historique
def train(model, trainloader, validloader, epochs=5, lr=0.001, earlystopping=False):
    optimizer = torch.optim.Adam(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()
    train_history, valid_history = [], []

    for epoch in range(epochs):
        model.train()  # Mode entraînement (dropout activé)
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Suivi de l'exactitude de l'entraînement
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Suivi de la perte et de l'exactitude de l'entraînement
        train_loss = running_loss / len(trainloader)
        train_acc = 100 * correct_train / total_train
        train_history.append((train_loss, train_acc))

        # Validation après chaque époque
        model.eval()  # Mode évaluation (dropout désactivé)
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0
        with torch.no_grad():
            for images, labels in validloader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()

        # Suivi de la perte et de l'exactitude de la validation
        valid_loss = valid_loss / len(validloader)
        valid_acc = 100 * correct_valid / total_valid
        valid_history.append((valid_loss, valid_acc))

        print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Epoch {epoch+1} - Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_acc:.2f}%")

        if earlystopping:
            # Implémentation possible pour early stopping
            pass

    return train_history, valid_history

# Visualisation des courbes de perte et de précision
def plot_train_val(train_history, valid_history):
    train_loss, train_acc = zip(*train_history)
    valid_loss, valid_acc = zip(*valid_history)

    # Tracer la courbe de perte
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train Loss')
    plt.plot(valid_loss, label='Valid Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Tracer la courbe de précision
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(valid_acc, label='Valid Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Charger le dataset Fashion-MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

dataset = torchvision.datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
trainset, validset = random_split(dataset, (50000, 10000))
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
validloader = DataLoader(validset, batch_size=64, shuffle=False)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

Ici, nous choisissons de définir un batch de taille 64...

Nous allons ensuite définir la classe du réseau de neurones qui n'utilise pas de dropout.

In [1]:
# Définition du modèle sans Dropout
class ReLUNet(nn.Module):
    def __init__(self):
        super(ReLUNet, self).__init__()
        self.fc1 = nn.Linear(784, 120)
        self.fc2 = nn.Linear(120,60)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

NameError: name 'nn' is not defined

La classe est définie, nous allons maintenant instancier un nouveau réseau, le tester et vérifier son erreur

In [ ]:
# instanciation et entraînement du modèle sans dropout
model = ReLUNet()
net_no_dropout = ReLUNet()  # Créer le modèle sans dropout
train_history_no_dropout, valid_history_no_dropout = train(
    net_no_dropout, 
    trainloader, validloader, 
    epochs=5, 
    lr=0.001
    )
plot_train_val(train_history_no_dropout, valid_history_no_dropout)

### Conclusion

Sans le dropout, le modèle a tendance à surapprendre, nous voyons alors que lors de la validation, l'erreur a tendance à grandir.
***
### Application du dropout

Nous allons ici étapes par étapes appliquer le dropout à notre modèle. Les différentes étapes seront présentées sous forme de questions auquelles vous pourrez tenter de répondre. Il est fortement conseillé de tenter de résoudre les questions par vous même  pour comprendre le dropout en pratique et savoir l'appliquer.

Torch utilise le dropout spatial, qui est décrit ici : https://arxiv.org/pdf/1411.4280.pdf

<div class="alert alert-success">
Question 1 : Commence par définir un modèle de réseau de neurones simple avec une couche de dropout. Cette couche doit être insérée après une couche entièrement connectée. Rappelle-toi que la couche Dropout prend un paramètre p qui définit la probabilité de désactivation des neurones. Je te conseille pour commencer de poser p=0.5 (50% de chance d'être désactivée.)
</div>

In [ ]:
# %load solutions/Q1.py
class ModelWithSingleDropout(nn.Module):
    def __init__(self):
        super(ModelWithSingleDropout, self).__init__()
        self.fc1 = nn.Linear(784, 120)
        
        ... # Créer la couche de dropout avec p=0.5
        
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        
        ... # Appliquer le dropout après la première couche
        
        x = self.fc2(x)
        return x

In [ ]:
# correction

class ModelWithSingleDropout(nn.Module):
    def __init__(self):
        super(ModelWithSingleDropout, self).__init__()
        self.fc1 = nn.Linear(784, 120)
        self.dropout = nn.Dropout(p=0.5)  # créer la couche de dropout avec p=0.5
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Appliquer le dropout après la première couche
        x = self.fc2(x)
        return x

Ici, nous avons ajouté une couche Dropout après la première couche linéaire. Cela permet de "désactiver" aléatoirement une partie des neurones de cette couche pendant l'entraînement.

<div class="alert alert-success">
Question 2 : Maintenant, modifie ton modèle en ajoutant plusieurs couches de dropout dans le réseau. Chaque couche devrait avoir une probabilité différente de désactivation des neurones. Par exemple, la première couche de dropout pourrait avoir p=0.3 et la deuxième p=0.5. Essaie d'appliquer des dropouts à différentes étapes du réseau.
</div>

In [ ]:
# %load solutions/Q2.py
class RandomDropoutModel(nn.Module):
    def __init__(self):
        ...
        # A compléter

    def forward(self, x):
        ...
        # A compléter

In [2]:
# correction

# Définition de la classe du modèle avec Dropout
class RandomDropoutModel(nn.Module):
    def __init__(self):
        super(RandomDropoutModel, self).__init__()
        self.dropout1 = nn.Dropout(p=0.2) # Dropout avec une probabilité de 20%
        self.fc1 = nn.Linear(784, 120)
        self.dropout2 = nn.Dropout(p=0.5)  # Dropout avec une probabilité de 50%
        self.fc2 = nn.Linear(120, 60)
        self.dropout3 = nn.Dropout(p=0.5)  # Dropout avec une probabilité de 50%
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.dropout1(x)  # Appliquer le premier dropout
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Appliquer le deuxième dropout
        x = F.relu(self.fc2(x))
        x = self.dropout3(x)  # Appliquer le troisième dropout
        x = self.fc3(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
# Instanciation et entraînement du modèle avec dropouts aléatoires
net = RandomDropoutModel()
train_history_with_dropout, valid_history_with_dropout = train(
    net, 
    trainloader, validloader, 
    epochs=5,
    lr=0.001,
    earlystopping=False)
plot_train_val(train_history_with_dropout, valid_history_with_dropout)

Maintenant à toi de jouer je te laisse créer un nouveau modèle de dropout avec les pourcentages que tu souhaites, essaye avec des petites et grandes valeurs pour bien visualiser l'effet du dropout.

<div class="alert alert-success">
Question 3 : Teste d'autres valeurs 
</div>

In [ ]:
# Définition de ton modèle avec Dropout
class YourDropoutModel(nn.Module):
    def __init__(self):
        super(YourDropoutModel, self).__init__()
        self.dropout1 = nn.Dropout(p=...)
        self.fc1 = nn.Linear(784, 120)
        self.dropout2 = nn.Dropout(p=...)
        self.fc2 = nn.Linear(120, 60)
        self.dropout3 = nn.Dropout(p=...)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.dropout1(x)  # Appliquer le premier dropout
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Appliquer le deuxième dropout
        x = F.relu(self.fc2(x))
        x = self.dropout3(x)  # Appliquer le troisième dropout
        x = self.fc3(x)
        return x

In [ ]:
# Instanciation et entraînement de ton modèle avec dropouts aléatoires
yournet = YourDropoutModel()
train_history_your_dropout, valid_history_your_dropout = train(
    yournet, 
    trainloader, validloader, 
    epochs=5,
    lr=..., # Ici tu peux le modifier pour visualiser son effet, 
# il est conseillé de l'augmenter légèrement par rapport au modèle sans dropout pour contrer le bruit généré
    earlystopping=False)
plot_train_val(train_history_your_dropout, valid_history_your_dropout)

Vous comprennez maintenant l'interêt de fixer des pourcentages précis :
- 20% de dropout dans les premières couches : garde une bonne quantité d'informations pour apprendre des représentations générales des données tout en réduisant le risque de surapprentissage.
- 50% de dropout dans les couches profondes : favorise la généralisation en forçant le modèle à apprendre des représentations plus robustes et moins spécifiques aux données d'entraînement.

Ainsi, la stratégie consiste à augmenter le dropout dans les couches profondes, où le modèle est déjà plus spécialisé, et à le garder modéré dans les couches initiales pour ne pas perdre des informations essentielles trop tôt.

### Bravo ! 
Vous avez appliqué le dropout au modèle de reconnaissance de chiffres de MNIST. J'espère que ca s'est bien passé et que vous avez bien saisi comment est ce que le dropout intervient dans l'entrainement pour éviter le surapprentissage. 

Je me permets de te transmettre un petit message du créateur de ce notebook :

*Salut à toi jeune codeur ! J'espère que ce petit message va te faire sourire au milieu de tes corrections de notebooks interminables. Je te souhaite bon courage pour la suite et surtout n'oublie pas de me mettre une bonne note, sinon je serais pas content. Bisous*
***

## 3. Comparer les performances d'un réseau avec et sans Dropout.

Le code suivant permet de comparer les performances des modèles précédents.


In [ ]:
# Créer la figure avec les 2 résultats précédents
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Tracer la courbe de perte (Loss)
axs[0].plot([x[0] for x in train_history_no_dropout], label='Train Loss (No Dropout)', color='blue')
axs[0].plot([x[0] for x in valid_history_no_dropout], label='Valid Loss (No Dropout)', color='red')
axs[0].plot([x[0] for x in train_history_with_dropout], label='Train Loss (With Dropout)', color='green')
axs[0].plot([x[0] for x in valid_history_with_dropout], label='Valid Loss (With Dropout)', color='orange')
axs[0].plot([x[0] for x in train_history_your_dropout], label='Train Loss (Your Model)', color='yellow')
axs[0].plot([x[0] for x in valid_history_your_dropout], label='Valid Loss (Your Model)', color='pink')
axs[0].set_title('Perte (Loss) par époque')
axs[0].set_xlabel('Époques')
axs[0].set_ylabel('Perte')
axs[0].legend()

# Tracer la courbe de précision (Accuracy)
axs[1].plot([x[1] for x in train_history_no_dropout], label='Train Accuracy (No Dropout)', color='blue')
axs[1].plot([x[1] for x in valid_history_no_dropout], label='Valid Accuracy (No Dropout)', color='red')
axs[1].plot([x[1] for x in train_history_with_dropout], label='Train Accuracy (With Dropout)', color='green')
axs[1].plot([x[1] for x in valid_history_with_dropout], label='Valid Accuracy (With Dropout)', color='orange')
axs[1].plot([x[1] for x in train_history_your_dropout], label='Train Accuracy (Your Model)', color='yellow')
axs[1].plot([x[1] for x in valid_history_your_dropout], label='Valid Accuracy (Your Model)', color='pink')
axs[1].set_title('Précision (Accuracy) par époque')
axs[1].set_xlabel('Époques')
axs[1].set_ylabel('Précision (%)')
axs[1].legend()

# Afficher le graphique
plt.tight_layout()
plt.show()


La il faut comparer les deux etc .....

Lorsqu'on va encore plus loin dans les epochs, l'effet des droppouts est encore plus visible, les temps de calculs sont longs donc je vais pas vous les faire faire mais voici les résultats que j'ai obtenu précédemment avec 50 epochs :

<img src= "img/resultats - p1=0,2 - p2=0,5 - p3=0,5 - 50 epochs.png" style="width: 900px; height: auto;">


***
## 4. Présenter cette technique sur d'autres types de modèles